In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, WeightedRandomSampler
from collections import Counter
from sklearn.metrics import classification_report

# --- Configuration ---
train_path = r"C:\Users\VEDIKA\Downloads\DATASET\train"
test_path = r"C:\Users\VEDIKA\Downloads\DATASET\test"
img_size = (224, 224)  # ResNet18 expects 224x224
batch_size = 32
num_classes = 4
emotions = ['happy', 'sad', 'angry', 'neutral']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Data Preparation ---
# Enhanced data augmentation for training
train_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=0.1),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=test_transform)

# Address class imbalance with WeightedRandomSampler
class_counts = Counter(train_dataset.targets)
total_samples = sum(class_counts.values())
class_weights = {i: total_samples / (num_classes * count) for i, count in class_counts.items()}
weights = [class_weights[label] for label in train_dataset.targets]
sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- Model Definition ---
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EmotionClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        # Freeze all layers except the last few
        for param in self.resnet.parameters():
            param.requires_grad = False
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
        # Replace the final layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

# Initialize model
model = EmotionClassifier(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# --- Training ---
def train_model(epochs=30):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader.dataset):.4f}, Val Accuracy: {val_acc:.2f}%")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "emotion_model_best.pt")
    
    print(f"Final Test Accuracy: {best_val_acc:.2f}%")
    
    # Detailed evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=emotions))

# Train the model
train_model()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
import cv2
import numpy as np
from PIL import Image
import webbrowser
from collections import deque
import logging
import os

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
img_size = (224, 224)
num_classes = 4
emotions = ['happy', 'sad', 'angry', 'neutral']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Spotify playlist URLs (public, no a01?thentication needed)
emotion_playlists = {
    'happy': 'https://open.spotify.com/playlist/37i9dQZF1DXdPec7aLTmlC',
    'sad': 'https://open.spotify.com/playlist/37i9dQZF1DX7qK8ma5wgG1',
    'angry': 'https://open.spotify.com/playlist/37i9dQZF1DX1tyCD9QhIWF',
    'neutral': 'https://open.spotify.com/playlist/37i9dQZF1DX4WYpdgoIcn6'
}

# Colors for display
emotion_colors = {
    'happy': (0, 255, 255),
    'sad': (255, 0, 255),
    'angry': (0, 0, 255),
    'neutral': (200, 200, 200)
}

# Sample weights for recommendation system to handle class imbalance
max_samples = 1185  # Based on largest class (sad)
sample_weights = {
    'happy': (max_samples / 162) * 2.0,   # ~14.63x
    'sad': max_samples / 1185,            # 1x
    'angry': (max_samples / 680) * 0.8,   # ~1.39x
    'neutral': (max_samples / 478) * 1.5  # ~3.72x
}

# Neural network model
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EmotionClassifier, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

# Recommendation system
class EmotionRecommender:
    def __init__(self, window_size=25, confidence_threshold=0.85):
        self.window = deque(maxlen=window_size)
        self.confidence_threshold = confidence_threshold
        self.feedback = {emotion: 1.0 for emotion in emotions}

    def add_prediction(self, emotion, confidence):
        if confidence >= self.confidence_threshold:
            self.window.append(emotion)
            logging.debug(f"Added prediction: {emotion} (confidence: {confidence:.2f})")

    def recommend_emotion(self):
        if not self.window:
            return None
        counts = {emotion: 0 for emotion in emotions}
        for emotion in self.window:
            counts[emotion] += sample_weights[emotion] * self.feedback[emotion]
        recommended = max(counts, key=counts.get)
        logging.debug(f"Recommended emotion: {recommended}")
        return recommended

    def apply_feedback(self, user_emotion, predicted_emotion):
        if user_emotion in emotions and predicted_emotion in emotions:
            self.feedback[user_emotion] = min(self.feedback.get(user_emotion, 1.0) + 0.5, 3.5)
            self.feedback[predicted_emotion] = max(self.feedback.get(predicted_emotion, 1.0) - 0.25, 0.2)
            logging.info(f"Feedback applied: {predicted_emotion} -> {user_emotion}")

def main():
    # Load model
    logging.info("Loading model...")
    model = EmotionClassifier(num_classes)
    try:
        model.load_state_dict(torch.load("emotion_model_best.pt", map_location=device))
        logging.info("Model loaded successfully")
    except Exception as e:
        logging.error(f"Error loading model: {e}")
        return
    
    model.eval()
    model.to(device)

    # Initialize webcam
    logging.info("Initializing webcam...")
    cap = None
    for index in [0, 1, 2]:
        cap = cv2.VideoCapture(index)
        if cap.isOpened():
            logging.info(f"Webcam opened successfully (index {index})")
            break
        cap.release()
    
    if not cap or not cap.isOpened():
        logging.error("Error: Could not open webcam...")
        return

    # Initialize face detector
    logging.info("Loading face cascade classifier...")
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    if face_cascade.empty():
        logging.error("Error: Could not load face cascade classifier.")
        cap.release()
        return
    
    # Initialize recommender
    recommender = EmotionRecommender(window_size=25, confidence_threshold=0.85)
    last_emotion = None
    
    # Preprocessing transform
    test_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    logging.info("Starting detection loop...")
    while True:
        ret, frame = cap.read()
        if not ret:
            logging.error("Error: Could not read frame from webcam.")
            break
        
        # Detect faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=15, minSize=(60, 60))
        
        for (x, y, w, h) in faces:
            # Extract and preprocess face
            face = frame[y:y+h, x:x+w]
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face_pil = Image.fromarray(face_rgb)
            face_tensor = test_transform(face_pil).unsqueeze(0).to(device)
            
            # Predict emotion
            with torch.no_grad():
                outputs = model(face_tensor)
                probabilities = torch.softmax(outputs, dim=1)
                confidence, predicted = torch.max(probabilities, 1)
                emotion = emotions[predicted.item()]
                confidence_val = confidence.item()
            
            # Add to recommender
            recommender.add_prediction(emotion, confidence_val)
            recommended_emotion = recommender.recommend_emotion() or emotion
            
            # Handle user feedback
            key = cv2.waitKey(1) & 0xFF
            if key == ord('h'):
                recommender.apply_feedback("happy", recommended_emotion)
                recommended_emotion = "happy"
            elif key == ord('s'):
                recommender.apply_feedback("sad", recommended_emotion)
                recommended_emotion = "sad"
            elif key == ord('a'):
                recommender.apply_feedback("angry", recommended_emotion)
                recommended_emotion = "angry"
            elif key == ord('n'):
                recommender.apply_feedback("neutral", recommended_emotion)
                recommended_emotion = "neutral"
            
            # Open Spotify playlist in browser
            if recommended_emotion != last_emotion and recommended_emotion is not None:
                last_emotion = recommended_emotion
                playlist_url = emotion_playlists.get(recommended_emotion)
                if playlist_url:
                    logging.info(f"Detected: {recommended_emotion} (Confidence: {confidence_val:.2f})")
                    try:
                        webbrowser.open(playlist_url)
                        logging.info(f"Opened {recommended_emotion} playlist in browser")
                    except Exception as e:
                        logging.error(f"Browser error: {e}")
            
            # Display
            color = emotion_colors[recommended_emotion]
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{recommended_emotion} ({confidence_val:.2f})", 
                        (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        cv2.imshow('Emotion Player', frame)
        
        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    logging.info("Cleaning up...")
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

2025-05-23 23:07:43,588 - INFO - Loading model...
2025-05-23 23:07:43,904 - INFO - Model loaded successfully
2025-05-23 23:07:43,906 - INFO - Initializing webcam...
2025-05-23 23:07:49,010 - INFO - Webcam opened successfully (index 0)
2025-05-23 23:07:49,010 - INFO - Loading face cascade classifier...
2025-05-23 23:07:49,051 - INFO - Starting detection loop...
2025-05-23 23:07:50,294 - INFO - Detected: neutral (Confidence: 0.47)
2025-05-23 23:07:50,421 - INFO - Opened neutral playlist in browser
2025-05-23 23:07:50,564 - INFO - Detected: angry (Confidence: 0.85)
2025-05-23 23:07:50,637 - INFO - Opened angry playlist in browser
2025-05-23 23:08:18,087 - INFO - Detected: happy (Confidence: 0.97)
2025-05-23 23:08:18,164 - INFO - Opened happy playlist in browser
2025-05-23 23:08:18,287 - INFO - Detected: sad (Confidence: 0.99)
2025-05-23 23:08:18,365 - INFO - Opened sad playlist in browser
2025-05-23 23:08:42,253 - INFO - Detected: angry (Confidence: 0.96)
2025-05-23 23:08:42,327 - INFO - 